In [3]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model and tokenizer
model_path = "D:/DATA FOR RESEARCH PROJECT/biobert_pacemaker_final-20250623T084209Z-1-001/biobert_pacemaker_final"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForSequenceClassification.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, use_safetensors=True)
model.eval()

# Flask app
app = Flask(__name__)
CORS(app)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "No text provided"}), 400

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs).item()
        conf = probs[0][pred].item()

    return jsonify({
        "prediction": "Adverse Event" if pred == 1 else "Not Adverse",
        "confidence": round(conf, 3)
    })

from flask import Flask, request

@app.route("/", methods=["GET"])
def home():
    return """
    <h2>BioBERT Adverse Event Classifier</h2>
    <p>Use the <code>/predict</code> endpoint via POST to get predictions.</p>
    <p>Example: send JSON <code>{"text": "Patient experienced pacing failure"}</code></p>
    """

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://100.91.93.250:5000
Press CTRL+C to quit
100.91.93.250 - - [23/Jun/2025 18:22:07] "GET / HTTP/1.1" 200 -


In [5]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model and tokenizer
model_path = "D:/DATA FOR RESEARCH PROJECT/biobert_pacemaker_final-20250623T084209Z-1-001/biobert_pacemaker_final"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForSequenceClassification.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, use_safetensors=True)
model.eval()

# Flask app
app = Flask(__name__)
CORS(app)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "No text provided"}), 400

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs).item()
        conf = probs[0][pred].item()

    return jsonify({
        "prediction": "Adverse Event" if pred == 1 else "Not Adverse",
        "confidence": round(conf, 3)
    })

from flask import Flask, request

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        text = request.form.get("text")
        if text:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
            with torch.no_grad():
                outputs = model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=1)
                pred = torch.argmax(probs).item()
                conf = probs[0][pred].item()
            return f"<h3>Prediction:</h3> {'Adverse Event' if pred == 1 else 'Not Adverse'}<br><b>Confidence:</b> {round(conf, 3)}"
    return '''
        <form method="post">
            <textarea name="text" rows="6" cols="60" placeholder="Paste FOI_TEXT here..."></textarea><br>
            <input type="submit">
        </form>
    '''

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://100.91.93.250:5000
Press CTRL+C to quit
100.91.93.250 - - [23/Jun/2025 19:09:20] "GET / HTTP/1.1" 200 -
100.91.93.250 - - [23/Jun/2025 19:10:30] "POST / HTTP/1.1" 200 -


In [2]:
# import requests

# url = "http://127.0.0.1:5000/predict"  # or use http://100.91.93.250:5000 if needed
# data = {"text": "The pacemaker battery failed during surgery."}

# response = requests.post(url, json=data)
# print(response.json())


In [1]:
# # Flask API after temperture scaling was done on biobert
# from flask import Flask, request, jsonify
# from flask_cors import CORS
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch
# import torch.nn.functional as F
# from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

# app = Flask(__name__)
# CORS(app)

# # 🔧 CONFIGURATION
# model_path = r"D:/DATA FOR RESEARCH PROJECT/flask API/deployed_model"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # 1. Load the config – this defines the model_type ("bert")
# config = AutoConfig.from_pretrained(model_path, local_files_only=True)

# # 2. Load the tokenizer using that config
# tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True, config=config)

# # 3. Load the classification model
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_path,
#     config=config,
#     local_files_only=True,
#     ignore_mismatched_sizes=True
# )
# model.to(device).eval()

# # Temperature scaler (must match saved version)
# class TemperatureScaler(torch.nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.temperature = torch.nn.Parameter(torch.tensor(1.0))  # 0-dim

#     def forward(self, logits):
#         return logits / self.temperature

# temp_scaler = TemperatureScaler().to(device)
# temp_scaler.load_state_dict(torch.load("temp_scaler.pth", map_location=device))
# temp_scaler.eval()

# # 🚀 PREDICTION ENDPOINT (JSON POST)
# @app.route('/predict', methods=['POST'])
# def predict():
#     data = request.get_json() or {}
#     text = data.get("text", "").strip()
#     if not text:
#         return jsonify({"error": "No text provided"}), 400

#     inputs = tokenizer(
#         text, return_tensors="pt", truncation=True, padding=True, max_length=512
#     ).to(device)

#     with torch.no_grad():
#         logits = model(**inputs).logits
#         scaled_logits = temp_scaler(logits)
#         probs = F.softmax(scaled_logits, dim=1)[0]
#         pred = torch.argmax(probs).item()
#         conf = probs[pred].item()

#     return jsonify({
#         "prediction": "Adverse Event" if pred == 1 else "Not Adverse",
#         "confidence": round(conf, 3)
#     })

# # 🖥 FORM-BASED UI (browser)
# @app.route("/", methods=["GET", "POST"])
# def home():
#     if request.method == "POST":
#         text = request.form.get("text", "").strip()
#         if text:
#             inputs = tokenizer(
#                 text, return_tensors="pt", truncation=True, padding=True, max_length=512
#             ).to(device)
#             with torch.no_grad():
#                 logits = model(**inputs).logits
#                 scaled_logits = temp_scaler(logits)
#                 probs = F.softmax(scaled_logits, dim=1)[0]
#                 pred = torch.argmax(probs).item()
#                 conf = probs[pred].item()

#             return f"""
#                 <h3>Prediction: {'Adverse Event' if pred == 1 else 'Not Adverse'}</h3>
#                 <b>Confidence:</b> {round(conf, 3)}
#                 <br><a href="/">Try again</a>
#             """
#     return '''
#         <form method="post">
#             <textarea name="text" rows="6" cols="60"
#                 placeholder="Paste FOI_TEXT here..."></textarea><br>
#             <input type="submit" value="Classify">
#         </form>
#     '''

# if __name__ == "__main__":
#     app.run(host="0.0.0.0", port=5000)


In [3]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

app = Flask(__name__)
CORS(app, origins=['http://localhost:3000'], methods=['POST'])
#CORS(app)

# CONFIGURATION
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = r"D:/DATA FOR RESEARCH PROJECT/biobert_pacemaker_final-20250623T084209Z-1-001/biobert_pacemaker_final"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

# Load model – ensure classifier head matches:
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    local_files_only=True,
    ignore_mismatched_sizes=True
)
model.to(device).eval()

# Temperature scaler (must match saved version)
class TemperatureScaler(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.temperature = torch.nn.Parameter(torch.tensor(1.0))  # 0-dim

    def forward(self, logits):
        return logits / self.temperature

temp_scaler = TemperatureScaler().to(device)
temp_scaler.load_state_dict(torch.load("D:/DATA FOR RESEARCH PROJECT/biobert_pacemaker_final-20250623T084209Z-1-001/biobert_pacemaker_final/temp_scaler.pth", map_location=device))
temp_scaler.eval()

# PREDICTION ENDPOINT (JSON POST)
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json() or {}
    text = data.get("text", "").strip()
    if not text:
        return jsonify({"error": "No text provided"}), 400

    inputs = tokenizer(
        text, return_tensors="pt", truncation=True, padding=True, max_length=512
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        scaled_logits = temp_scaler(logits)
        probs = F.softmax(scaled_logits, dim=1)[0]
        pred = torch.argmax(probs).item()
        conf = probs[pred].item()

    return jsonify({
        "prediction": "Adverse Event" if pred == 1 else "Not Adverse",
        "confidence": round(conf, 3)
    })

# FORM-BASED UI (browser)
@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        text = request.form.get("text", "").strip()
        if text:
            inputs = tokenizer(
                text, return_tensors="pt", truncation=True, padding=True, max_length=512
            ).to(device)
            with torch.no_grad():
                logits = model(**inputs).logits
                scaled_logits = temp_scaler(logits)
                probs = F.softmax(scaled_logits, dim=1)[0]
                pred = torch.argmax(probs).item()
                conf = probs[pred].item()

            return f"""
                <h3>Prediction: {'Adverse Event' if pred == 1 else 'Not Adverse'}</h3>
                <b>Confidence:</b> {round(conf, 3)}
                <br><a href="/">Try again</a>
            """
    return '''
        <form method="post">
            <textarea name="text" rows="6" cols="60"
                placeholder="Paste FOI_TEXT here..."></textarea><br>
            <input type="submit" value="Classify">
        </form>
    '''

@app.route('/health', methods=['GET'])
def health():
    return jsonify(status="ok", model_version="v1.0"), 200

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.150:5000
Press CTRL+C to quit
192.168.1.150 - - [13/Jul/2025 21:41:13] "GET / HTTP/1.1" 200 -
192.168.1.150 - - [13/Jul/2025 21:41:13] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.150 - - [13/Jul/2025 21:44:00] "POST / HTTP/1.1" 200 -
192.168.1.150 - - [13/Jul/2025 21:44:03] "GET / HTTP/1.1" 200 -


In [36]:
import os
print("CWD:", os.getcwd())
print("Files here:", os.listdir())


CWD: C:\Users\samridhi\Research Project part A
Files here: ['.ipynb_checkpoints', '0.26.0', "0.26.0'", 'API.ipynb', 'app_csv.ipynb', 'BertPacemaker.ipynb', 'EDA and Text analysis.ipynb', 'false_negatives.csv', 'false_positives.csv', 'flagged_adverse_cases.csv', 'my_plot.png', 'PacemakerAnalysis.ipynb', 'pacemakerClean-checkpoint.ipynb', 'Random Forest Classifier.ipynb', 'ResProject1-checkpoint.ipynb', 'results', 'Statisical Analysis.ipynb', 'Trend analysis.ipynb']


In [40]:
state_dict_path = r"D:/DATA FOR RESEARCH PROJECT/biobert_pacemaker_final-20250623T084209Z-1-001/biobert_pacemaker_final/temp_scaler.pth"
print("Exists?", os.path.exists(state_dict_path))
temp_scaler.load_state_dict(torch.load(state_dict_path, map_location=device))


Exists? True


<All keys matched successfully>